In [1]:
from pysmt.shortcuts import Symbol, And, Equals, Int, Or, Implies, Solver, Not, EqualsOrIff
from pysmt.typing import INT
import itertools
import numpy as np

In [2]:
N = np.array([
    [0] + [1]*3 + [2]*6,
    [0]*2 + [1]*3 + [2] + [3]*2 + [2]*2,
    [0]*2 + [4]*2 + [5]*2 + [6, 3] + [7]*2,
    [0]*2 + [8, 4, 9] + [6]*3 + [7]*2,
    [0] + [8]*2 + [9]*2 + [10, 11] + [6]*2 + [7],
    [0, 12, 8, 13, 14, 10] + [15]*2 + [7]*2,
    [0, 16, 17] + [13]*3 + [15] + [20]*3,
    [16]*2 + [17, 18, 13, 19, 22] + [21]*2 + [22],
    [16]*3 + [18]*2 + [22]*2 + [21]*2 + [22],
    [16]*4 + [18]*2 + [22]*4
])

In [3]:
R = range(10)

f = True
s = [[Symbol(f"{i},{j}", INT) for j in R] for i in R]
for i, j in itertools.product(R, R):
        f &= (1 <= s[i][j]) & (s[i][j] <= int((N == N[i][j]).sum()))
f

(((((... & ...) & (... & ...)) & ((... <= ...) & (... <= ...))) & ((1 <= '9,8') & ('9,8' <= 9))) & ((1 <= '9,9') & ('9,9' <= 9)))

In [4]:
def eq(i, j, k):
    return Equals(s[i][j], Int(k))
def neq(i, j, k):
    return Not(eq(i, j, k)) 
    
for i, j in itertools.product(R, R):
    for i_, j_ in itertools.product(R, R):
        if (i, j) < (i_, j_) and N[i][j] == N[i_][j_]:
            f &= Not(Equals(s[i][j], s[i_][j_]))

In [5]:
for i, j in itertools.product(range(10), range(10)):
    for k in range(1, N[i][j] + 1):
        ok, nok = [], []
        for i_, j_ in itertools.product(range(10), range(10)):
            if abs(i - i_) + abs(j - j_) == k:
                ok.append(eq(i_, j_, k))
            if abs(i - i_) + abs(j - j_) < k and (i, j) != (i_, j_):
                nok.append(neq(i_, j_, k))
        f &= Implies(eq(i, j, k), And(Or(ok), And(nok)))

In [6]:
for (i, j, k) in [(0, 1, 3), (0, 5, 7), (1, 3, 4), (2, 8, 2), (3, 3, 1), (4, 0, 6), (4, 2, 1), (5, 7, 3), (5, 9, 6), (6, 6, 2), (7, 1, 2), (8, 6, 6), (9, 4, 5), (9, 8, 2)]:
    f &= eq(i, j, k)

In [7]:
with Solver() as solver:
    solver.add_assertion(f)
    if solver.solve():
        V = [[solver.get_value(s[i][j]).constant_value() for j in R] for i in R]
        for i in range(10):
            for j in range(10):
                print(V[i][j], "" if V[i][j] == 10 else " ", end=" ")
            print()


9   3   6   2   3   7   4   9   6   5   
8   7   5   4   1   1   2   3   8   2   
10  5   3   2   1   2   5   1   2   4   
4   1   2   1   2   6   3   1   1   3   
6   3   1   1   3   2   1   4   2   7   
2   1   4   5   1   1   1   3   5   6   
3   1   2   3   2   4   2   1   2   3   
7   2   1   1   1   1   3   1   4   9   
5   8   3   4   2   1   6   2   3   8   
4   6   9   10  5   3   4   7   2   5   


In [8]:
from functools import reduce
from operator import mul
sum(reduce(mul, V[i]) for i in R)

16842072